In [2]:
from astropy.cosmology import FlatLambdaCDM
from time import clock
from os import path
import glob, os
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.miles_util as lib
from astropy.table import Table, Column

In [3]:
def flux2L(flux,z):
    """Transfer flux to luminoity assuming a flat Universe"""
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    DL = cosmo.luminosity_distance(z).value*10**6*3.08*10**18 # unit cm
    L = flux*1.e-17*4.*np.pi*DL**2   #erg/s/A
    return L

def L_conti(wave,flux,z):
    return np.asarray([np.log10(5100*flux2L(flux[np.where( abs(wave - 5100.) < 5.,True,False)].mean(),z) )])

In [4]:
tie_balmer=False
limit_doublets=False
ppxf_dir = path.dirname(path.realpath(ppxf_package.__file__))

mass_frac_all = np.array([])
ml_all = np.array([])
L_all = np.array([])
zbin2_syst_vel = []

In [5]:
for i in zip(glob.glob('/Users/ANguyen\MyWorks\zbin2_specs_sv\spec-*.fits')):
    #print (np.shape(i))
    hdu = fits.open(i[0])
    t = hdu[1].data
    wv = 10**t['loglam']
    redshift = float(hdu[2].data['z'])
    #print ('redshift:',np.round(redshift,4))
    
    wave1 = 10**t['loglam']/(1.+ redshift)
    flux1 = t['flux']
    err1 = 1./np.sqrt(t['ivar'])
    z = 0.
    mask = np.where( (wave1 > 3541.) & (wave1 < 7409.) & (err1 <100.), True, False)

    flux = flux1[mask]
    galaxy = flux/np.median(flux)   # Normalize spectrum to avoid numerical issues
    factor1=np.median(flux)
    #print 'flux normalized by:', np.median(flux)
    wave = wave1[mask]
    err = err1[mask]
    wave *= np.median(util.vac_to_air(wave)/wave)
    noise = np.full_like(galaxy, 0.01635)  # Assume constant noise per pixel here
    c = 299792.458  # speed of light in km/s
    velscale = c*np.log(wave[1]/wave[0])  # eq.(8) of Cappellari (2017)
    FWHM_gal = 2.76  # SDSS has an approximate instrumental resolution FWHM of 2.76A.
    pathname = ppxf_dir + '/miles_models/Mun1.30*.fits'
    miles = lib.miles(pathname, velscale, FWHM_gal)
    reg_dim = miles.templates.shape[1:]
    stars_templates = miles.templates.reshape(miles.templates.shape[0], -1)

    regul_err = 0.01  # Desired regularization error

    lam_range_gal = np.array([np.min(wave), np.max(wave)])/(1. + z)
    gas_templates, gas_names, line_wave = util.emission_lines(
        miles.log_lam_temp, lam_range_gal, FWHM_gal,
        tie_balmer=tie_balmer, limit_doublets=limit_doublets)
    templates = np.column_stack([stars_templates, gas_templates])

    c = 299792.458
    dv = c*(miles.log_lam_temp[0] - np.log(wave[0]))  # eq.(8) of Cappellari (2017)
    vel = c*np.log(1 + z)   # eq.(8) of Cappellari (2017)
    start = [vel, 180.]  # (km/s), starting guess for [V, sigma]

    n_temps = stars_templates.shape[1]
    n_forbidden = np.sum(["[" in a for a in gas_names])  # forbidden lines contain "[*]"
    n_balmer = len(gas_names) - n_forbidden
    component = [0]*n_temps + [1]*n_balmer + [2]*n_forbidden
    gas_component = np.array(component) > 0  # gas_component=True for gas templates
    moments = [4, 4, 4]

    start = [start, start, start]

    gas_reddening = 0 if tie_balmer else None
    t = clock()
    
    len_galaxy = len(galaxy)
    len_template = len(templates)
    difference = len_galaxy - len_template

    if len_galaxy > len_template:
        galaxy = galaxy[:len(galaxy)-difference]
        noise = noise[:len(noise)-difference]
        wave = noise[:len(wave)-difference]
        
    elif len_galaxy == len_template:
        galaxy = galaxy[:len(galaxy)-1]
        noise = noise[:len(noise)-1]
        wave = noise[:len(wave)-1]
    else:
        galaxy = galaxy
        noise = noise
        
    pp = ppxf(templates, galaxy, noise, velscale, start,
              plot=False, moments=moments, degree=-1, mdegree=10, vsyst=dv,
              lam=wave, clean=False, regul=1./regul_err, reg_dim=reg_dim,
              component=component, gas_component=gas_component,
              gas_names=gas_names, gas_reddening=gas_reddening)
    
    syst_vel = c*np.log(1 + redshift) + pp.sol[1][0]
    
    print ('syst_vel',syst_vel)
    
    zbin2_syst_vel.append(syst_vel)
    
    #print('Desired Delta Chi^2: %.4g' % np.sqrt(2*galaxy.size))
    #print('Current Delta Chi^2: %.4g' % ((pp.chi2 - 1)*galaxy.size))

      

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -104        63     0.015     0.001
 comp.  1:      -174        51    -0.058    -0.010
 comp.  2:      -110        68     0.006     0.085
chi2/DOF: 215.9; degree = -1; mdegree = 10
method = capfit; Jac calls: 10; Func calls: 235; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  35 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.4956     0.037    -174    51
Comp:  1        Hgamma       1.11     0.037    -174    51
Comp:  1         Hbeta      2.757     0.035    -174    51
Comp:  1        Halpha      8.291     0.032    -174    51
Comp:  2     [OII]3726      1.681     0.046    -110    68
Comp:  2     [OII]3729      7.617     0.046    -110    68
Comp:  2     [SII]6716      1.676     0.036    -110    68
Comp:  2     [SII]6731      1.232     0.036    -110   

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       123       195     0.076    -0.005
 comp.  1:       197       209    -0.107     0.027
 comp.  2:       104       204    -0.072     0.106
chi2/DOF: 286.2; degree = -1; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 166; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  26 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      2.302     0.056     197   209
Comp:  1        Hgamma      5.212     0.056     197   209
Comp:  1         Hbeta      10.69     0.056     197   209
Comp:  1        Halpha       27.5     0.055     197   209
Comp:  2     [OII]3726      61.09     0.097     104   204
C

...

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:       128       135     0.065     0.053
 comp.  1:       281       121    -0.081     0.087
 comp.  2:       237       158    -0.148     0.083
chi2/DOF: 14.58; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 209; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  46 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta        0.3     0.047     281   121
Comp:  1        Hgamma     0.5852     0.047     281   121
Comp:  1         Hbeta      1.281     0.046     281   121
Comp:  1        Halpha      7.593     0.045     281   121
Comp:  2     [OII]3726      3.616     0.074     237   158
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        81       171     0.057    -0.007
 comp.  1:       201       109    -0.060    -0.069
 comp.  2:       141       118     0.024     0.028
chi2/DOF: 15.42; degree = -1; mdegree = 10
method = capfit; Jac calls: 8; Func calls: 186; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  45 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.3786     0.042     201   109
Comp:  1        Hgamma      0.441     0.042     201   109
Comp:  1         Hbeta      1.879     0.041     201   109
Comp:  1        Halpha      9.982     0.039     201   109
Comp:  2     [OII]3726      3.167     0.058     141   118
C

Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -145        83     0.005     0.011
 comp.  1:      -160       129    -0.039    -0.071
 comp.  2:      -158       194     0.030     0.212
chi2/DOF: 161.2; degree = -1; mdegree = 10
method = capfit; Jac calls: 11; Func calls: 257; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  10 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      1.135     0.044    -160   129
Comp:  1        Hgamma       1.97     0.044    -160   129
Comp:  1         Hbeta      4.184     0.043    -160   129
Comp:  1        Halpha      33.11     0.042    -160   129
Comp:  2     [OII]3726          0       0.1    -158   194


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -116       143     0.045    -0.120
 comp.  1:      -208       216    -0.051    -0.072
 comp.  2:      -170       174    -0.053    -0.110
chi2/DOF: 251.7; degree = -1; mdegree = 10
method = capfit; Jac calls: 11; Func calls: 258; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  26 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta      1.451     0.054    -208   216
Comp:  1        Hgamma     0.5178     0.054    -208   216
Comp:  1         Hbeta      1.687     0.053    -208   216
Comp:  1        Halpha      15.94     0.053    -208   216
Comp:  2     [OII]3726          0     0.068    -170   174


Emission lines included in gas templates:
['Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[OII]3726' '[OII]3729' '[SII]6716'
 '[SII]6731' '[OIII]5007_d' '[OI]6300_d' '[NII]6583_d']
 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -127       204     0.032    -0.091
 comp.  1:      -160       213     0.029    -0.047
 comp.  2:      -233       150     0.023     0.043
chi2/DOF: 88.23; degree = -1; mdegree = 10
method = capfit; Jac calls: 9; Func calls: 211; Status: 2
linear_method = lsqbox; Nonzero Templates (>0.1%):  55 / 161
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component   name       flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1        Hdelta     0.5522     0.055    -160   213
Comp:  1        Hgamma          0     0.054    -160   213
Comp:  1         Hbeta      0.663     0.054    -160   213
Comp:  1        Halpha       2.31     0.053    -160   213
Comp:  2     [OII]3726          0      0.07    -233   150
C

In [6]:
zbin2_syst_vel_array = np.array(zbin2_syst_vel)

In [7]:
col0 = fits.Column(name = 'SYST_VEL', format = 'D', array = zbin2_syst_vel_array)

coldefs = fits.ColDefs([col0])
hdu = fits.BinTableHDU.from_columns(coldefs)

zbin2_sv = Table(hdu.data)

In [8]:
(zbin2_sv).write('zbin2_sv.fits')